In [ ]:
# import flask related
from flask import Flask, request, abort
# import linebot related
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
    LocationSendMessage, ImageSendMessage, StickerSendMessage,ImageMessage, AudioMessage
)

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.operations import ComputerVisionClientOperationsMixin
from azure.cognitiveservices.vision.computervision.models import TextOperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import time

import os
import speech_recognition as sr
import azure.cognitiveservices.speech as speechsdk

#連結linebot
# create flask server
app = Flask(__name__)
# your linebot message API - Channel access token (from LINE Developer)
line_bot_api = LineBotApi(Channel access token)
# your linebot message API - Channel secret
handler = WebhookHandler(Channel secret)

#圖片辨識api key
# Set API key.
subscription_key = 'API key'
# Set endpoint.
endpoint = 'https://computervisioncourse.cognitiveservices.azure.com/'
url = 'https://westus.api.cognitive.microsoft.com/vision/v3.2-preview.1/read/analyze?language=zh-Hans'

# Call API
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

#語音辨識api
def transcribe(wav_path):
    # Creates an instance of a speech config with specified subscription key and service region.
    # Replace with your own subscription key and region identifier from here: https://aka.ms/speech/sdkregion

    speech_key, service_region = "your_speech_key", "southcentralus"
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

    # Creates an audio configuration that points to an audio file.
    # Replace with your own audio filename.
    
    audio_input = speechsdk.audio.AudioConfig(filename=wav_path)

    # Creates a recognizer with the given settings
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input,language='zh-TW')

    print("Recognizing first result...")
    result = speech_recognizer.recognize_once()
    return result.text

#接收使用者傳送的訊息
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        print('receive msg')
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'

#回傳使用說明
# handle msg
@handler.add(MessageEvent, message=TextMessage)
def handle_image(event):
  
    # get user info & message
    user_id = event.source.user_id
    msg = event.message.text
    user_name = line_bot_api.get_profile(user_id).display_name
    
    # push text_msg
    line_bot_api.push_message(user_id,
                              TextSendMessage(text = user_name+'您好^^'))
    
    # get msg details
    print('msg from [', user_name, '](', user_id, ') : ', msg)
    
    line_bot_api.reply_message(event.reply_token, 
                               [TextSendMessage(text = '請上傳您的筆記照片或音檔'),
                                StickerSendMessage(package_id='4',sticker_id='626')])
#回傳圖片辨識結果    
@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message (event):
    if event.message.type=='image':
        message_content = line_bot_api.get_message_content(event.message.id)        
        
        #把使用者上傳圖片寫入指定位置
        file_path = 'C:/Users/DA01014/Desktop/picforazure/image.jpg'
        with open(file_path, 'wb') as fd:
            for chunk in message_content.iter_content():
                fd.write(chunk)
        '''
        Batch Read File, recognize printed text - remote
        This example will extract printed text in an image, then print results, line by line.
        This API call can also recognize handwriting (not shown).
        '''

        # Get an image with printed handwritten
        #remote_image_printed_text_url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/printed_handwritten.pdf"

        # Call API with URL and raw response (allows you to get the operation location)
        #recognize_printed_results = computervision_client.batch_read_file(remote_image_printed_text_url,  raw=True)

        # 指定圖檔
        local_image_path = file_path

        # 讀取圖片
        local_image = open(local_image_path, "rb")
        #params = {'language':'zh-Hant','detectOrientation':'true'}
        #recognize_printed_results = computervision_client.batch_read_file_in_stream(local_image,raw=True)
        recognize_printed_results = computervision_client.recognize_printed_text_in_stream(local_image,detect_orientation=True, language='unk')
        
        file_name = ""
        
        #辨識圖片內容文字
        for region in recognize_printed_results.regions:
            text_list=[]
            for line in region.lines:
                s=" "
                for word in line.words:
                    s+=word.text+' '
                if 'ref' in s: #附加小功能 如果圖片上標示ref.檔名(以前上傳的) linebot回傳辨識內容後再回傳該檔案圖片
                    file_name=s.split('ref.')[1]
                    print(file_name)
                text_list.append(s)
                
                #print(s)
        alltext='\n'.join(text_list)
        print()   
        #line_bot_api.reply_message(event.reply_token, TextSendMessage(text = s)) 
        
        reply_arr=[]
#         print(type(file_name.strip()))
#        print(file_name.strip() =='note2.jpg')
        
        #如果圖片上有標示舊檔名(ref.filename),回傳辨識內容及舊檔圖片
        #這邊也是demo用,把它寫死,所以舊檔名預設為note2
        if file_name.strip() =='note2.jpg':
            textsend = reply_arr.append(TextSendMessage(text = alltext))    
            textsend2= reply_arr.append(TextSendMessage(text='This is your reference document.'))
            imagesend = reply_arr.append(ImageSendMessage(original_content_url='https://info.bigindustries.be/hs-fs/hubfs/Schermafbeelding%202018-08-09%20om%2011.28.37.png?width=669&name=Schermafbeelding%202018-08-09%20om%2011.28.37.png', preview_image_url='https://info.bigindustries.be/hs-fs/hubfs/Schermafbeelding%202018-08-09%20om%2011.28.37.png?width=669&name=Schermafbeelding%202018-08-09%20om%2011.28.37.png'))
        #
        else :
            textsend = reply_arr.append(TextSendMessage(text = alltext))    
            #textsend2= reply_arr.append(TextSendMessage(text='This is your reference document.'))
            #imagesend = reply_arr.append(ImageSendMessage(original_content_url='https://info.bigindustries.be/hs-fs/hubfs/Schermafbeelding%202018-08-09%20om%2011.28.37.png?width=669&name=Schermafbeelding%202018-08-09%20om%2011.28.37.png', preview_image_url='https://info.bigindustries.be/hs-fs/hubfs/Schermafbeelding%202018-08-09%20om%2011.28.37.png?width=669&name=Schermafbeelding%202018-08-09%20om%2011.28.37.png'))
        
        
        line_bot_api.reply_message(event.reply_token, reply_arr) 
       
#回傳語音辨識內容

@handler.add(MessageEvent, message=AudioMessage)
def handle_audio(event):

    name_mp3 = 'recording.mp3'
    name_wav = 'recording.wav'
    
    message_content = line_bot_api.get_message_content(event.message.id)
    
    #開啟使用者上傳音檔
    with open(name_mp3, 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)
    #linebot語音訊息為wav檔，用ffmpeg將檔案轉為mp3格式
    os.system('ffmpeg -y -i ' + name_mp3 + ' ' + name_wav + ' -loglevel quiet')
    #os.system('ffmpeg -y -i ' + name_m4a + ' ' + name_wav)
    
    #call語音辨識function
    text = transcribe('C:/Users/DA01014/Desktop/' + name_wav)
    
    #text = transcribe('C:/Users/DA01014/Desktop/Azure/azure_台北班/Module_11_語音服務/narration.wav')
    print('Transcribe:', text)
    #將辨識結果傳回給使用者
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text = text))
        
# run app
if __name__ == "__main__":
    app.run(host='127.0.0.1', port=12345)